# New nlpfuncs for hathi/digitizedwork

In [ ]:
%run ppa_init.ipynb
%run ppa_nlpfuncs.ipynb

## Querying for matches

In [ ]:
class WordQuery:
    
    def __init__(self,word='',words=[]):
        if word: words+=[word]
        self._words = words
        self._works = []
        self._counts_df = None
        self._qset = None
    
    def get_queryset(self):
        if self._qset is None:
            self._qset=WorkWordCounts.objects.filter(data__has_any_keys=self.get_words())
        return self._qset

    def get_works(self):
        return [wwc.work for wwc in self.get_queryset()]
    
    def get_words(self):
        return self._words
    
    def get_counts(self):
        o = []
        iterr = self.get_queryset()
        for wwc in tqdm(iterr):
            work = wwc.work
            countd = wwc.data
            total = sum(countd.values())
            for word in self.get_words():
                count = countd.get(word,0)
                odx = dict(
                    work=work.id,
                    word=word,
                    count=count,
                    total=total,
                    fpm=count/total*10**6
                )
                o.append(odx)
        self._counts_df = pd.DataFrame(o).sort_values('fpm',ascending=False)        
        return self._counts_df
    
    def get_meta(self, keys = ['title','pub_date']):
        o = []
        for work in self.get_works():
            d = work.__dict__
            odx = {k:d.get(k,'') for k in keys}
            o.append(dict(work=work.id, **odx))
        return pd.DataFrame(o)
    

In [ ]:
wq = WordQuery('ballad')
wq.get_queryset()

In [ ]:
wq.get_meta()

In [ ]:
wq.get_counts()

In [ ]:
wq.get_counts().merge(wq.get_meta(), on='work', how='left')

In [ ]:
# @TODO: Is there no way to separate other languages?
german_work = DigitizedWork.objects.get(id=3051)
german_work.title
german_work.get_txt()

In [ ]:
def get_lang(self):
    from langdetect import detect

    
    


def test_get_lang():
    from langdetect import detect

    german_works = {3051}
    for id in german_works:
        work = DigitizedWork.objects.get(id=id)
        txt = work.get_txt()
        print(work, work.title, detect(txt))
        assert detect(txt) == 'de'
        

In [ ]:
test_get_lang()

In [ ]:
!pip install langdetect

In [ ]:
from langdetect import detect
detect("Hello world")

In [ ]:
detect("Hallo Welt")